In [47]:
import requests
from pprint import pprint

# Example GET request to the API root
get_url = 'https://www.aagwa.org/api'
response = requests.get(get_url)
print('GET status:', response.status_code)
try:
    pprint(response.json())
except Exception as e:
    print('Response is not JSON:', response.text)

# Example POST request (if the API supports it)
# Replace 'endpoint' and 'payload' with actual values as needed
post_url = 'https://www.aagwa.org/api'
payload = {'example_key': 'example_value'}
response = requests.post(post_url, json=payload)
print('POST status:', response.status_code)
try:
    pprint(response.json())
except Exception as e:
    print('Response is not JSON:', response.text)

GET status: 404
Response is not JSON: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL was not found on this server.</p>
<hr>
<address>Apache/2.4.58 (Win64) OpenSSL/3.1.3 PHP/7.4.33 mod_fcgid/2.3.10-dev Server at www.aagwa.org Port 443</address>
</body></html>

POST status: 404
Response is not JSON: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL was not found on this server.</p>
<hr>
<address>Apache/2.4.58 (Win64) OpenSSL/3.1.3 PHP/7.4.33 mod_fcgid/2.3.10-dev Server at www.aagwa.org Port 443</address>
</body></html>



In [48]:
#Retrieving list of Countries
countries_url = 'https://www.aagwa.org/api/countries'
response = requests.get(countries_url)
print('GET status:', response.status_code)
try:
    pprint(response.json())
except Exception as e:
    print('Response is not JSON:', response.text)

GET status: 200
[{'country': 'Uganda', 'id': '0ac40c0b-22f5-4e9a-a6da-bca701779184'},
 {'country': 'Malawi', 'id': '275e8738-080c-11f0-9201-0b48478dee03'},
 {'country': 'Ghana', 'id': '7056b1ba-0319-11f0-9558-0e95c4172471'},
 {'country': 'Benin', 'id': 'b2708d41-6eda-4b94-b32a-d653a54f17a3'},
 {'country': 'Senegal', 'id': 'f64b3ba1-03e5-45c3-afe6-8a4caa64e1ff'}]


In [49]:
#Retrieving Information on the Map of Uganda using the
map_url = 'https://www.aagwa.org/api/predictions?country=Uganda&feature=production'
response = requests.get(map_url)
data = response.json()
print(response)


<Response [200]>


###Working With FS CORS data

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as g


In [87]:
calcium_df = pd.read_csv('./Calcium.csv')

calcium_df.head()

,Category,Average consumption adequacy of Kilocalories (kcal) (NAME_1),Average consumption adequacy of Kilocalories (kcal) (value)
0,0,ABIM,66.99
1,1,ABIM,66.99
2,2,ADJUMANI,22.91
3,3,AGAGO,64.41
4,4,AGAGO,64.41


In [52]:
zinc = pd.read_csv('./Zinc.csv')
zinc.head()

,Category,Average consumption adequacy of Kilocalories (kcal) (NAME_1),Average consumption adequacy of Kilocalories (kcal) (value)
0,0,ABIM,70.86
1,1,ABIM,70.86
2,2,ADJUMANI,63.57
3,3,AGAGO,86.78
4,4,AGAGO,86.78


In [53]:
# List of Nutrients - Load all CSVs, set a consistent 'district' column,
# rename the last (value) column to the requested descriptive name, and drop duplicates by district
import glob
import os

csv_files = [
    'Calcium.csv', 'Foliate.csv', 'Iron.csv', 'Kilocaleries.csv',
    'Proteins.csv', 'Riboflavin.csv', 'Thiamin.csv', 'VitaminA.csv', 'VitaminB12.csv',
    'VitaminB6.csv', 'VitaminC.csv', 'Zinc.csv',
]

# Units (as provided) and pretty display names where needed
units = {
    'Calcium': 'mg',
    'Foliate': 'mcg',      # assuming Foliate -> Folate
    'Niacin': 'mg',        # not present in csv_files; kept for completeness
    'Proteins': 'mg',
    'Kilocaleries': 'kcal',
    'Iron': 'mg',
    'Riboflavin': 'mg',
    'Thiamin': 'mg',
    'VitaminA': 'mcg',
    'VitaminB12': 'mcg',
    'VitaminB6': 'mg',
    'VitaminC': 'mg',
    'Zinc': 'mg',
}

# Friendly display names (for nicer column titles)
display_name = {
    'Calcium': 'Calcium',
    'Foliate': 'Folate',
    'Proteins': 'Proteins',
    'Kilocaleries': 'Kilocaleries',
    'Iron': 'Iron',
    'Riboflavin': 'Riboflavin',
    'Thiamin': 'Thiamin',
    'VitaminA': 'Vitamin A',
    'VitaminB12': 'Vitamin B12',
    'VitaminB6': 'Vitamin B6',
    'VitaminC': 'Vitamin C',
    'Zinc': 'Zinc',
}

nutrient_dfs = {}
missing_files = []
for file in csv_files:
    if not os.path.exists(file):
        missing_files.append(file)
        continue
    df = pd.read_csv(file)
    # Ensure we have at least two columns (district and value)
    if df.shape[1] < 2:
        nutrient_dfs[file.replace('.csv','')] = df
        continue
    # Robustly identify the district column: prefer columns with 'name' or 'district' in the header, else use column index 1
    district_col = None
    for col in df.columns:
        lname = str(col).lower()
        if 'district' in lname or 'name' in lname:
            district_col = col
            break
    if district_col is None and len(df.columns) >= 2:
        district_col = df.columns[1]
    # Rename the district column to 'district'
    if district_col is not None:
        df = df.rename(columns={district_col: 'district'})
    # Build last-column display name based on filename base
    base = file.replace('.csv','')
    pretty = display_name.get(base, base)
    unit = units.get(base, '')
    # Compose the requested column name
    value_col_new = f'Average Consumption adequacy of {pretty} ({unit})' if unit else f'Average Consumption adequacy of {pretty}'
    # Rename the last column (whatever it currently is) to the composed name
    last_col = df.columns[-1]
    # If last_col is already 'district' (rare), try to find the numeric/value column instead
    if last_col == 'district' and df.shape[1] >= 2:
        last_col = df.columns[-1]  # fallback - keeps being last
    df = df.rename(columns={last_col: value_col_new})
    # Drop duplicates by district if the district column exists
    if 'district' in df.columns:
        df = df.drop_duplicates(subset=['district']).reset_index(drop=True)
    nutrient_dfs[base] = df

# Report missing files (if any) and show a sample head for verification
print('Missing files skipped:', missing_files)
if 'Calcium' in nutrient_dfs:
    nutrient_dfs['Calcium'].head()

Missing files skipped: []


In [54]:
#Obtaining dataframes
for df_name, df in nutrient_dfs.items():
    print(f"DataFrame for {df_name}:")
    print(df.head())

DataFrame for Calcium:
   Category  district  Average Consumption adequacy of Calcium (mg)
0         0      ABIM                                         66.99
1         2  ADJUMANI                                         22.91
2         3     AGAGO                                         64.41
3         5  ALEBTONG                                         63.12
4         7  AMOLATAR                                         45.78
DataFrame for Foliate:
   Category  district  Average Consumption adequacy of Folate (mcg)
0         0      ABIM                                         84.26
1         2  ADJUMANI                                         67.69
2         3     AGAGO                                         97.48
3         5  ALEBTONG                                         93.26
4         7  AMOLATAR                                         98.96
DataFrame for Iron:
   Category  district  Average Consumption adequacy of Iron (mg)
0         0      ABIM                                

In [55]:
# Merge safe: reduce each dataframe to ['district', '<nutrient_value>'] where the value column is detected robustly
valid_dfs = {k: v for k, v in nutrient_dfs.items() if isinstance(v, pd.DataFrame) and 'district' in v.columns}
print('DataFrames found for merge:', list(valid_dfs.keys()))

if not valid_dfs:
    print('No valid DataFrames with a "district" column available to merge.')
else:
    cleaned = {}
    for name, df in valid_dfs.items():
        # columns except 'district'
        other_cols = [c for c in df.columns if c != 'district']
        if not other_cols:
            print(f'Skipping {name}: no value column found')
            continue
        # Prefer a column containing the 'Average Consumption adequacy' phrase
        val_col = None
        for c in other_cols:
            if 'Average Consumption adequacy' in str(c):
                val_col = c
                break
        # Fallback to numeric columns
        if val_col is None:
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            numeric_cols = [c for c in numeric_cols if c != 'district']
            if numeric_cols:
                val_col = numeric_cols[0]
        # Final fallback: last non-district column
        if val_col is None:
            val_col = other_cols[-1]
        # Compose a clean, unique column name for this nutrient
        pretty = display_name.get(name, name)
        unit = units.get(name, '')
        new_col = f'Average Consumption adequacy of {pretty} ({unit})' if unit else f'Average Consumption adequacy of {pretty}'
        # Select and rename
        cleaned_df = df[['district', val_col]].copy()
        cleaned_df = cleaned_df.rename(columns={val_col: new_col})
        # Drop duplicates by district
        cleaned_df = cleaned_df.drop_duplicates(subset=['district']).reset_index(drop=True)
        cleaned[name] = cleaned_df
    # Now merge all cleaned dfs on 'district'
    from functools import reduce
    dfs_to_merge = list(cleaned.values())
    if not dfs_to_merge:
        print('No cleaned DataFrames to merge')
    else:
        merged_nutrients = reduce(lambda left, right: pd.merge(left, right, on='district', how='outer'), dfs_to_merge)
        merged_nutrients = merged_nutrients.sort_values('district').reset_index(drop=True)
        nutrient_dfs['merged_by_district'] = merged_nutrients
        print('Merged shape:', merged_nutrients.shape)
        merged_nutrients.head()

DataFrames found for merge: ['Calcium', 'Foliate', 'Iron', 'Kilocaleries', 'Proteins', 'Riboflavin', 'Thiamin', 'VitaminA', 'VitaminB12', 'VitaminB6', 'VitaminC', 'Zinc']
Merged shape: (122, 13)


In [56]:
merged_nutrients

,district,Average Consumption adequacy of Calcium (mg),Average Consumption adequacy of Folate (mcg),Average Consumption adequacy of Iron (mg),Average Consumption adequacy of Kilocaleries (kcal),Average Consumption adequacy of Proteins (mg),Average Consumption adequacy of Riboflavin (mg),Average Consumption adequacy of Thiamin (mg),Average Consumption adequacy of Vitamin A (mcg),Average Consumption adequacy of Vitamin B12 (mcg),Average Consumption adequacy of Vitamin B6 (mg),Average Consumption adequacy of Vitamin C (mg),Average Consumption adequacy of Zinc (mg)
0,ABIM,66.99,84.26,75.84,69.68,81.61,64.11,86.52,87.13,31.12,85.43,92.35,70.86
1,ADJUMANI,22.91,67.69,41.55,67.63,100.00,55.10,98.38,95.76,100.00,100.00,94.15,63.57
2,AGAGO,64.41,97.48,76.41,76.30,89.48,86.58,90.74,96.13,71.50,96.51,100.00,86.78
3,ALEBTONG,63.12,93.26,58.17,70.04,67.80,67.91,84.71,87.76,36.47,90.96,100.00,62.15
4,AMOLATAR,45.78,98.96,54.54,80.77,64.28,65.99,91.42,87.75,42.76,99.06,99.34,55.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,SSEMBABULE,69.75,98.65,70.40,88.81,89.47,86.86,92.35,85.88,45.22,100.00,100.00,78.24
118,TORORO,80.38,95.82,77.85,88.52,86.77,88.55,91.63,91.24,59.60,96.21,97.68,79.34
119,WAKISO,66.90,97.66,69.73,87.04,90.80,85.33,92.06,90.45,73.56,97.97,95.90,76.03
120,YUMBE,42.23,85.98,40.99,53.85,56.82,57.40,59.70,59.95,42.80,64.98,82.97,46.29


In [57]:
merged_nutrients.isna().sum()

,0
district,0
Average Consumption adequacy of Calcium (mg),0
Average Consumption adequacy of Folate (mcg),0
Average Consumption adequacy of Iron (mg),0
Average Consumption adequacy of Kilocaleries (kcal),0
Average Consumption adequacy of Proteins (mg),0
Average Consumption adequacy of Riboflavin (mg),0
Average Consumption adequacy of Thiamin (mg),0
Average Consumption adequacy of Vitamin A (mcg),0
Average Consumption adequacy of Vitamin B12 (mcg),0


In [58]:
merged_nutrients.duplicated().sum()

np.int64(0)

In [59]:
merged_nutrients.rename(columns={'Average Consumption adequacy of Kilocaleries (kcal)': 'Consumption adequacy of Kilocaleries (kcal)'}, inplace=True)

## Average  Consumption adequacy of Nutrients
   Average Consumption Adequacy of a nutrient is a measure of how much of that nutrient people in a given population are consuming compared to the recommended requirement.

If the adequacy is 100% → the population meets the recommended intake.

If it is less than 100% → there is a deficiency risk.

If it is greater than 100% → intake exceeds the requirement


$\text{Adequacy (%)} = \frac{\text{Average intake of nutrient}}{\text{Recommended daily requirement}} \times 100$


In [60]:
merged_nutrients

,district,Average Consumption adequacy of Calcium (mg),Average Consumption adequacy of Folate (mcg),Average Consumption adequacy of Iron (mg),Consumption adequacy of Kilocaleries (kcal),Average Consumption adequacy of Proteins (mg),Average Consumption adequacy of Riboflavin (mg),Average Consumption adequacy of Thiamin (mg),Average Consumption adequacy of Vitamin A (mcg),Average Consumption adequacy of Vitamin B12 (mcg),Average Consumption adequacy of Vitamin B6 (mg),Average Consumption adequacy of Vitamin C (mg),Average Consumption adequacy of Zinc (mg)
0,ABIM,66.99,84.26,75.84,69.68,81.61,64.11,86.52,87.13,31.12,85.43,92.35,70.86
1,ADJUMANI,22.91,67.69,41.55,67.63,100.00,55.10,98.38,95.76,100.00,100.00,94.15,63.57
2,AGAGO,64.41,97.48,76.41,76.30,89.48,86.58,90.74,96.13,71.50,96.51,100.00,86.78
3,ALEBTONG,63.12,93.26,58.17,70.04,67.80,67.91,84.71,87.76,36.47,90.96,100.00,62.15
4,AMOLATAR,45.78,98.96,54.54,80.77,64.28,65.99,91.42,87.75,42.76,99.06,99.34,55.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,SSEMBABULE,69.75,98.65,70.40,88.81,89.47,86.86,92.35,85.88,45.22,100.00,100.00,78.24
118,TORORO,80.38,95.82,77.85,88.52,86.77,88.55,91.63,91.24,59.60,96.21,97.68,79.34
119,WAKISO,66.90,97.66,69.73,87.04,90.80,85.33,92.06,90.45,73.56,97.97,95.90,76.03
120,YUMBE,42.23,85.98,40.99,53.85,56.82,57.40,59.70,59.95,42.80,64.98,82.97,46.29


In [61]:
merged_nutrients.columns

Index(['district', 'Average Consumption adequacy of Calcium (mg)',
       'Average Consumption adequacy of Folate (mcg)',
       'Average Consumption adequacy of Iron (mg)',
       'Consumption adequacy of Kilocaleries (kcal)',
       'Average Consumption adequacy of Proteins (mg)',
       'Average Consumption adequacy of Riboflavin (mg)',
       'Average Consumption adequacy of Thiamin (mg)',
       'Average Consumption adequacy of Vitamin A (mcg)',
       'Average Consumption adequacy of Vitamin B12 (mcg)',
       'Average Consumption adequacy of Vitamin B6 (mg)',
       'Average Consumption adequacy of Vitamin C (mg)',
       'Average Consumption adequacy of Zinc (mg)'],
      dtype='object')

### Exploring Districts with the lowest adequacy in various nutrients in Uganda

### Calcium

In [63]:

nutrient = "Average Consumption adequacy of Calcium (mg)"
bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Folate

In [64]:

nutrient =    "Average Consumption adequacy of Folate (mcg)"
bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Iron

In [65]:

nutrient ="Average Consumption adequacy of Iron (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Kilocaleries

In [66]:

nutrient =  "Consumption adequacy of Kilocaleries (kcal)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Proteins

In [67]:

nutrient =  "Average Consumption adequacy of Proteins (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Riboflavin

In [68]:

nutrient ="Average Consumption adequacy of Riboflavin (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Thiamin

In [69]:

nutrient = "Average Consumption adequacy of Thiamin (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Vitamin A

In [70]:

nutrient = "Average Consumption adequacy of Vitamin A (mcg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Vitamin B12

In [71]:

nutrient ="Average Consumption adequacy of Vitamin B12 (mcg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Vitamin B6

In [72]:

nutrient ="Average Consumption adequacy of Vitamin B6 (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Vitamin C

In [73]:

nutrient ="Average Consumption adequacy of Vitamin C (mg)"

bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Zinc

In [74]:

nutrient = "Average Consumption adequacy of Zinc (mg)"
bottom5 = merged_nutrients.nsmallest(5, nutrient)
bottom5["Group"] = "Bottom 5"

# Horizontal bar chart
fig = px.bar(
    bottom5,
    x=nutrient,
    y="district",
    color="Group",
    text=nutrient,
    orientation="h",
     color_discrete_map={
         "Bottom 5": "red"
          }
)

fig.update_traces(textposition="outside")
fig.update_layout(
    title=f"Bottom 5 Districts by {nutrient}",
    xaxis_title="Adequacy",
    yaxis_title="District",
    yaxis=dict(categoryorder="total ascending")
)

fig.show()


### Most vulnarable districts
From the charts above we see districts such as **Lamwo**, **Nyowa**, **Adjumani**, **Pallisa** and **Buteebo** which appear several times among  the bottom five districts with the least avarage consumption adequacy of nutrients and kilocalories.
This puts the people of these distrcits; most of which are in northren Uganda at a very high risk of the dangers of malnutrion which include,Stunted growth, dangerously low weight, weak immunity, higher risk of death,

### Hidden Hunger
Hidden hunger refers to a form of malnutrition caused by micronutrient deficiencies lack of essential vitamins and minerals such as iron, zinc, vitamin A, and iodine despite having sufficient calorie intake. In other words, individuals may eat enough calories, but their diets lack the nutrient quality needed for healthy bodily function

Even when calories are adequate, diets can be nutritionally poor hidden hunger captures this gap between quantity and quality.

It's a silent threat, often overlooked but deeply damaging to health and development.

In [86]:
fig = px.scatter(merged_nutrients,
                 x='Consumption adequacy of Kilocaleries (kcal)',
                 y="Average Consumption adequacy of Iron (mg)",
                 title="Hidden Hunger: Calories vs Iron Adequacy",
                 hover_data='district')
fig.show()

For instance in the scatterplot above districts such as Butebo, Rukungiri and Mityana have very consumption adequacy of kilocalories but have very low consumption adequacy of Iron; a vital nutrient for immunity and proper brain activity in a human.
These danger can be ignored because the people in these districts can look healthy due to their adequate calorie intake despite them lacking vital nutrients